In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

### Examining the Data

In [2]:
# Let's start with reading the data 

users_data = pd.read_pickle("../data/users_data_final.pkl")

In [3]:
# look at a preview of the data

users_data.head(n = 3)

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,False,True,False,None,261,1,-1,0,None,False,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,NaN,NaN,44,technician,single,secondary,False,True,False,None,151,1,-1,0,None,False,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,False,True,True,None,76,1,-1,0,None,False,2002-06-04,mobile


In [4]:
# look at the shape of the data (rows, columns)

users_data.shape

(45209, 19)

In [5]:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45209 entries, 0 to 45215
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              45209 non-null  object        
 1   number_transactions  35210 non-null  float64       
 2   total_amount_usd     35210 non-null  float64       
 3   age                  45209 non-null  int64         
 4   job                  44921 non-null  object        
 5   marital              45209 non-null  object        
 6   education            43352 non-null  object        
 7   default              45209 non-null  bool          
 8   housing              45209 non-null  bool          
 9   loan                 45209 non-null  bool          
 10  contact              32191 non-null  object        
 11  duration             45209 non-null  int64         
 12  campaign             45209 non-null  int64         
 13  pdays                45209 non-

### Feature Engineering: Handling Missing Data

In [6]:
# identify missing values

def missing_dataframe(df):
    """
    @param df pandas DataFrame
    @return a DataFrame with the percentage of missing data for every feature
    """
    
    percent_missing = df.isnull().mean() 
    
    missing_value_df = pd.DataFrame(percent_missing).reset_index() # convert to DataFrame
    missing_value_df = missing_value_df.rename(columns = {"index" : "feature",
                                                                0 : "percent_missing"}) # rename columns

    missing_value_df = missing_value_df.sort_values(by = ['percent_missing'], ascending = False) # sort the values
    
    return missing_value_df

missing_value_df = missing_dataframe(users_data)
missing_value_df

,feature,percent_missing
15,poutcome,0.817470
10,contact,0.287952
2,total_amount_usd,0.221173
1,number_transactions,0.221173
6,education,0.041076
4,job,0.006370
18,device,0.002079
17,date_joined,0.000664
12,campaign,0.000000
16,term_deposit,0.000000


In [7]:
# drop pouctome as it is missing 82% data

users_data.drop(['poutcome'], axis = 1, inplace = True)

In [8]:
# replace missing categorical values with "Unknown"

users_data['job'].fillna("Unknown", inplace = True)
users_data['education'].fillna("Unknown", inplace = True)
users_data['contact'].fillna("Unknown", inplace = True)
users_data['device'].fillna("Unknown", inplace = True)


In [9]:
# replace missing numerical values with 0

users_data['number_transactions'].fillna(0, inplace = True)
users_data['total_amount_usd'].fillna(0, inplace = True)

In [10]:
# drop rows where date joined is null

users_data_cleaned = users_data[~users_data.date_joined.isnull()]

In [11]:
users_data_cleaned.isnull().sum()

# there should be no missing data

user_id                0
number_transactions    0
total_amount_usd       0
age                    0
job                    0
marital                0
education              0
default                0
housing                0
loan                   0
contact                0
duration               0
campaign               0
pdays                  0
previous               0
term_deposit           0
date_joined            0
device                 0
dtype: int64

### Feature Engineering: date_joined 

In [12]:
# Did you come up with a way to engineer date joined variable?
# if not drop this variable

users_data_cleaned.drop(['date_joined'], axis = 1, inplace = True)


/Users/shaq/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Feature Engineering: Categorical Data

In [13]:
def identify_number_categories(df):
    """This definition can be used 
    
    @param df DataFrame 
    
    @param returns a DataFrame with the categorical features and number of categories"""

    categorical_columns = list(df.select_dtypes(['object']).columns)

    cat_df = []

    for c in categorical_columns:
        cat_df.append({"categorical_feature": c,
                       "number_categories": len(df[c].value_counts(dropna = False))
                    })
        
    return pd.DataFrame(cat_df)

In [14]:
cat_df = identify_number_categories(users_data_cleaned)
cat_df

,categorical_feature,number_categories
0,user_id,45179
1,job,12
2,marital,3
3,education,4
4,contact,3
5,device,4


In [15]:
# drop catgeorical features where number_categories is greater than 10
features_to_drop = list(cat_df[cat_df.number_categories > 10].categorical_feature)
print(features_to_drop)

['user_id', 'job']


In [16]:
users_data_cleaned.drop(features_to_drop, axis = 1, inplace = True)

/Users/shaq/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
# check the variables were dropped

list(users_data_cleaned)

['number_transactions',
 'total_amount_usd',
 'age',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'term_deposit',
 'device']

In [18]:
# identify all the categorical variables 

categorical_columns = list(users_data_cleaned.select_dtypes(['object']).columns)

categorical_columns

['marital', 'education', 'contact', 'device']

In [19]:
# encode the categorical variables
# lots of ways to do that, but we will use pd.get_dummies function
# lets start with marital
# we use pandas get dummies function

pd.get_dummies(users_data_cleaned['marital'], prefix='marital')

# you can also one of dummy variables, as it can be inferred by the other dummy variables

,marital_divorced,marital_married,marital_single
0,0,1,0
1,0,0,1
2,0,1,0
3,0,1,0
4,0,0,1
...,...,...,...
45211,0,1,0
45212,1,0,0
45213,0,1,0
45214,0,1,0


In [20]:
def one_hot(df, categorical_cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    
    for c in categorical_cols:
        dummies = pd.get_dummies(df[c], prefix=c)
        df = pd.concat([df, dummies], axis=1)
        df.drop(c, axis = 1, inplace = True)
    
    return df

In [21]:
# encode all of the categorical columns

categorical_columns = list(users_data_cleaned.select_dtypes(['object']).columns)

users_data_cleaned_w_encoding = one_hot(users_data_cleaned, categorical_columns)

In [22]:
# preview the data

users_data_cleaned_w_encoding.head(n = 5)

,number_transactions,total_amount_usd,age,default,housing,loan,duration,campaign,pdays,previous,...,education_primary,education_secondary,education_tertiary,contact_Unknown,contact_cellular,contact_telephone,device_Unknown,device_desktop,device_mobile,device_tablet
0,3.0,2143.0,58,False,True,False,261,1,-1,0,...,0,0,1,1,0,0,0,0,1,0
1,0.0,0.0,44,False,True,False,151,1,-1,0,...,0,1,0,1,0,0,0,1,0,0
2,2.0,2.0,33,False,True,True,76,1,-1,0,...,0,1,0,1,0,0,0,0,1,0
3,0.0,0.0,47,False,True,False,92,1,-1,0,...,0,0,0,1,0,0,0,0,0,1
4,1.0,1.0,33,False,False,False,198,1,-1,0,...,0,0,0,1,0,0,0,0,1,0


In [23]:
print(users_data_cleaned.shape)
print(users_data_cleaned_w_encoding.shape)

# you should have ten new columns

(45179, 15)
(45179, 25)


In [24]:
users_data_cleaned_w_encoding.info()

# Check all the data is numerical
# If not, do not continue

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45179 entries, 0 to 45215
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   number_transactions  45179 non-null  float64
 1   total_amount_usd     45179 non-null  float64
 2   age                  45179 non-null  int64  
 3   default              45179 non-null  bool   
 4   housing              45179 non-null  bool   
 5   loan                 45179 non-null  bool   
 6   duration             45179 non-null  int64  
 7   campaign             45179 non-null  int64  
 8   pdays                45179 non-null  int64  
 9   previous             45179 non-null  int64  
 10  term_deposit         45179 non-null  bool   
 11  marital_divorced     45179 non-null  uint8  
 12  marital_married      45179 non-null  uint8  
 13  marital_single       45179 non-null  uint8  
 14  education_Unknown    45179 non-null  uint8  
 15  education_primary    45179 non-null 

### Building our Linear Regression Model

In [25]:
# Step 1. Define the target and features
# A supervised machine learning algorithm requires both - uses historical data to uncover relationships between other features of your dataset and the target.

target_data = users_data_cleaned_w_encoding["total_amount_usd"]
features = users_data_cleaned_w_encoding.drop(["total_amount_usd"], axis = 1)


In [26]:
# Step 2. Split the data into a training and test set
# The training data is the data we use to train the machine learning algorithm
# the test set is used to evaluate the prediction

# using this handy function from scikit-learm to split the data into a training and test dataset
# we can adjust the test size to our needs, but it's best practise to train the model on 70 - 80% of the data

X_train, X_test, y_train, y_test = train_test_split(
                                                features, 
                                                target_data,
                                                test_size = 0.2, 
                                                random_state = 42)


In [27]:
# print the shape of the training data

print("Training Data")
print(X_train.shape)
print(y_train.shape)

Training Data
(36143, 24)
(36143,)


In [28]:
# print the shape of the test data 

print("\nTest Data")
print(X_test.shape)
print(y_test.shape)


Test Data
(9036, 24)
(9036,)


In [29]:
# Step 3. Import the Logistic Regression model from sklearn

lr = LinearRegression()

In [30]:
# Step 4. Fit the model on the training data

lr.fit(X_train, y_train) 

# This is your machine learning model!

LinearRegression()

In [31]:
# Step 5. Make predictions on unseen data

y_pred = lr.predict(X_test) 

In [32]:
print("first five predicted total amounts:", y_pred[0:5])
print("first five actual total amounts:", list(y_test[0:5]))

first five predicted total amounts: [1573.21790537 1212.91978156 1041.89517373 1034.82594881 1824.75392547]
first five actual total amounts: [3025.0, 197.0, 5057.0, 378.0, 1230.0]


In [33]:
# Step 6: Evaluate the model performance

r2_score(y_test, y_pred)

# R^2 (coefficient of determination) regression score function.
# Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). 

# Pretty close to 0. Lets see over the next few weeks we can improve the score.


0.024612036929752668

In [34]:
mean_absolute_error(y_test, y_pred)

1285.3917918833747

In [35]:
mean_squared_error(y_test, y_pred)

6898203.039929008